In [1]:
import os
from nibabel import cifti2
import numpy as np
import itertools
import nibabel as nib
import argparse

aws_access_key_id='...'
aws_secret_access_key='....'

import boto3
import sys
import pickle

/home/nicogravel/anaconda3/envs/py36/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%%javascript
function toggler(){
    if(window.already_toggling){
        // Don't add multiple buttons.
        return 0
    }
    let btn = $('.input').append('<button>Toggle Code</button>')
        .children('button');
    btn.on('click', function(e){
        let tgt = e.currentTarget;
        $(tgt).parent().children('.inner_cell').toggle()
    })
    window.already_toggling = true;
}
// Since javascript cells are executed as soon as we load
// the notebook (if it's trusted), and this cell might be at the
// top of the notebook (so is executed first), we need to
// allow time for all of the other code cells to load before
// running. Let's give it 5 seconds.

setTimeout(toggler, 5000);

<IPython.core.display.Javascript object>

In [3]:
fileobj     = open('subj_id.txt')
data        = fileobj.read()
subject_ID  = data.split('\n')
print('Subjects : ', subject_ID)

task_ID_1  = ['tfMRI_RETBAR1_7T_AP',
              'tfMRI_RETBAR2_7T_PA'
              ]

task_ID_2  = ['tfMRI_MOVIE1_7T_AP',
              'tfMRI_MOVIE2_7T_PA',
              'tfMRI_MOVIE3_7T_PA',
              'tfMRI_MOVIE4_7T_AP', 
             ]

task_ID_3  = ['rfMRI_REST1_7T_PA',
              'rfMRI_REST2_7T_AP',
              'rfMRI_REST3_7T_PA',
              'rfMRI_REST4_7T_AP'
             ]

task_ID = task_ID_1 + task_ID_2 + task_ID_3
print('Number of tasks : ',len(task_ID))

temp_root = '/scratch/nicogravel/HCP_data/temp/'
file      = '_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'

Subjects :  ['100610', '102311', '102816', '104416', '105923', '108323', '109123', '111312', '111514', '114823', '115017', '115825', '116726', '118225', '125525', '126426', '128935', '130518', '131217', '131722', '134627', '134829', '135124', '137128', '140117', '144226', '145834', '146129', '146432', '146735', '146937', '148133', '150423', '155938', '156334', '157336', '158035', '158136', '159239', '162935', '164131', '164636', '165436', '167036', '167440', '169343', '169747', '171633', '172130', '173334', '175237', '176542', '177140', '177645', '177746', '178142', '178243', '178647', '180533', '181232', '181636', '182436', '182739', '185442', '186949', '187345', '191033', '191336', '191841', '192439', '192641', '193845', '195041', '196144', '197348', '198653', '199655', '200210', '200311', '200614', '201515', '203418', '204521', '205220', '209228', '212419', '214019', '214524', '221319', '233326', '239136', '246133', '249947', '251833', '257845', '263436', '283543', '318637', '320826

In [4]:
n_subj = 0
x_subj = 0
for subj in range(len(subject_ID)):
    isdir = os.path.isdir(temp_root + subject_ID[subj] + '/')   
    #print(isdir)   
    if isdir == True: 
        folder = temp_root + subject_ID[subj]          
        # get size
        size = 0
        for ele in os.scandir(folder):
            size+=os.path.getsize(ele)
        print('subject: ', subject_ID[subj], ', size(MB): ',  size/1024**2)  
        n_subj = + n_subj + 1
    if isdir == False: 
        x_subj = + x_subj + 1
print('Subject downloaded : ',n_subj, ',  Subject to download : ', x_subj)


subject:  100610 , size(MB):  5123.201560974121
subject:  102311 , size(MB):  5123.201560974121
subject:  102816 , size(MB):  5123.201560974121
subject:  104416 , size(MB):  5123.201560974121
subject:  105923 , size(MB):  5123.201560974121
subject:  108323 , size(MB):  5123.201560974121
subject:  109123 , size(MB):  5123.201560974121
subject:  111312 , size(MB):  4536.369209289551
subject:  111514 , size(MB):  5120.202827453613
subject:  114823 , size(MB):  5120.202827453613
subject:  115017 , size(MB):  5120.202827453613
subject:  115825 , size(MB):  5120.202827453613
subject:  116726 , size(MB):  5120.202827453613
subject:  118225 , size(MB):  5120.202827453613
subject:  125525 , size(MB):  5120.202827453613
subject:  126426 , size(MB):  5120.202827453613
subject:  128935 , size(MB):  5120.202827453613
subject:  130518 , size(MB):  5120.202827453613
subject:  131217 , size(MB):  5120.202827453613
subject:  131722 , size(MB):  5120.202827453613
subject:  134627 , size(MB):  5120.20282

In [4]:
n_subj = 0
x_subj = 0

for subj in range(len(subject_ID)):
    isdir = os.path.isdir(temp_root + subject_ID[subj] + '/')   
  
    
    if isdir == True: 
        folder = temp_root + subject_ID[subj]          
        # get size
        size = 0
        for ele in os.scandir(folder):
            size+=os.path.getsize(ele)
        print('subject: ', subject_ID[subj], ', size(MB): ',  size/1024**2)  
        n_subj = + n_subj + 1
        
    if isdir == False: 
        os.mkdir(temp_root + subject_ID[subj])
        for run in range(len(task_ID)):
            print('Processing data for subject :', subject_ID[subj], 'task :', task_ID[run])
            dbhcp = 'HCP_1200/' + subject_ID[subj] + '/MNINonLinear/Results/'
            s3_client = boto3.client('s3',
                                    aws_access_key_id = aws_access_key_id,
                                    aws_secret_access_key = aws_secret_access_key)
            s3_bucket_name = 'hcp-openaccess'
            s3_path = dbhcp + task_ID[run] + '/' + task_ID[run] + file
            print(s3_path)
            temp_save_path = temp_root + subject_ID[subj] + '/' + task_ID[run] + file
            print(temp_save_path)
            print('Downloading...')
            s3_client.download_file(Bucket=s3_bucket_name, Key=s3_path, Filename=temp_save_path)
            sub_file = temp_save_path

subject:  100610 , size(MB):  5123.201560974121
subject:  102311 , size(MB):  5123.201560974121
subject:  102816 , size(MB):  5123.201560974121
subject:  104416 , size(MB):  5123.201560974121
subject:  105923 , size(MB):  5123.201560974121
subject:  108323 , size(MB):  5123.201560974121
subject:  109123 , size(MB):  5123.201560974121
subject:  111312 , size(MB):  4536.369209289551
subject:  111514 , size(MB):  5120.202827453613
subject:  114823 , size(MB):  5120.202827453613
subject:  115017 , size(MB):  5120.202827453613
subject:  115825 , size(MB):  5120.202827453613
subject:  116726 , size(MB):  5120.202827453613
subject:  118225 , size(MB):  5120.202827453613
subject:  125525 , size(MB):  5120.202827453613
subject:  126426 , size(MB):  5120.202827453613
subject:  128935 , size(MB):  5120.202827453613
subject:  130518 , size(MB):  5120.202827453613
subject:  131217 , size(MB):  5120.202827453613
subject:  131722 , size(MB):  5120.202827453613
subject:  134627 , size(MB):  5120.20282

Processing data for subject : 826353 task : tfMRI_MOVIE3_7T_PA
HCP_1200/826353/MNINonLinear/Results/tfMRI_MOVIE3_7T_PA/tfMRI_MOVIE3_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/826353/tfMRI_MOVIE3_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 826353 task : tfMRI_MOVIE4_7T_AP
HCP_1200/826353/MNINonLinear/Results/tfMRI_MOVIE4_7T_AP/tfMRI_MOVIE4_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/826353/tfMRI_MOVIE4_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 826353 task : rfMRI_REST1_7T_PA
HCP_1200/826353/MNINonLinear/Results/rfMRI_REST1_7T_PA/rfMRI_REST1_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/826353/rfMRI_REST1_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 826353 task : rfMRI_REST2_7T_AP
HCP_1200/826353/MNINonLinear/Resu

HCP_1200/861456/MNINonLinear/Results/tfMRI_RETBAR2_7T_PA/tfMRI_RETBAR2_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/861456/tfMRI_RETBAR2_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 861456 task : tfMRI_MOVIE1_7T_AP
HCP_1200/861456/MNINonLinear/Results/tfMRI_MOVIE1_7T_AP/tfMRI_MOVIE1_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/861456/tfMRI_MOVIE1_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 861456 task : tfMRI_MOVIE2_7T_PA
HCP_1200/861456/MNINonLinear/Results/tfMRI_MOVIE2_7T_PA/tfMRI_MOVIE2_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/861456/tfMRI_MOVIE2_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 861456 task : tfMRI_MOVIE3_7T_PA
HCP_1200/861456/MNINonLinear/Results/tfMRI_MOVIE3_7T_PA/tfMRI_MOVIE3_7T_PA_Atlas_1.6mm_M

Processing data for subject : 872764 task : rfMRI_REST4_7T_AP
HCP_1200/872764/MNINonLinear/Results/rfMRI_REST4_7T_AP/rfMRI_REST4_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/872764/rfMRI_REST4_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 878776 task : tfMRI_RETBAR1_7T_AP
HCP_1200/878776/MNINonLinear/Results/tfMRI_RETBAR1_7T_AP/tfMRI_RETBAR1_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/878776/tfMRI_RETBAR1_7T_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 878776 task : tfMRI_RETBAR2_7T_PA
HCP_1200/878776/MNINonLinear/Results/tfMRI_RETBAR2_7T_PA/tfMRI_RETBAR2_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
/scratch/nicogravel/HCP_data/temp/878776/tfMRI_RETBAR2_7T_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii
Downloading...
Processing data for subject : 878776 task : tfMRI_MOVIE1_7T_AP
HCP_1200/878776/MNINonLi